In [52]:
# Install Python dependencies: Flask, OpenCV, YOLOv8
!pip install --quiet flask opencv-python-headless ultralytics


In [53]:
%%bash
# Create the folder structure
mkdir -p app/templates app/static

# index.html (upload form)
cat > app/templates/index.html << 'EOF'
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>People Counter</title>
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <link rel="stylesheet"
        href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"/>
</head>
<body class="min-h-screen bg-gradient-to-r from-blue-200 via-purple-200 to-pink-200
             flex items-center justify-center p-4">
  <div class="bg-white bg-opacity-80 rounded-2xl p-8 shadow-xl w-full max-w-sm
              animate__animated animate__fadeInUp">
    <h1 class="text-3xl font-bold text-gray-800 text-center mb-6">👥 People Counter</h1>
    {% if error %}
      <p class="text-red-600 text-center mb-4">{{ error }}</p>
    {% endif %}
    <form action="/" method="post" enctype="multipart/form-data" class="space-y-4">
      <input type="file" name="file" accept="image/*" required
             class="block w-full text-gray-700 file:py-2 file:px-4
                    file:rounded-full file:bg-indigo-600 file:text-white
                    hover:file:bg-indigo-700 transition"/>
      <button type="submit"
              class="w-full py-2 bg-indigo-600 hover:bg-indigo-700 text-white font-semibold
                     rounded-full transform hover:scale-105 transition">
        Upload & Detect
      </button>
    </form>
  </div>
</body>
</html>
EOF

# result.html (only annotated image + count)
cat > app/templates/result.html << 'EOF'
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Detection Result</title>
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <link rel="stylesheet"
        href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"/>
</head>
<body class="min-h-screen bg-gradient-to-r from-blue-200 via-purple-200 to-pink-200
             flex items-center justify-center p-4">
  <div class="bg-white bg-opacity-90 rounded-2xl p-8 shadow-xl w-full max-w-md
              animate__animated animate__zoomIn">
    <h2 class="text-2xl font-bold text-gray-800 mb-6">👀 People Detected: {{ count }}</h2>
    <img src="{{ annotated_url }}" alt="Annotated"
         class="mx-auto rounded-lg shadow-md w-full mb-6"/>
    <a href="/" class="inline-block px-6 py-2 bg-indigo-600 hover:bg-indigo-700 text-white
                       font-semibold rounded-full transform hover:rotate-3 transition">
      Try Another
    </a>
  </div>
</body>
</html>
EOF


In [55]:
%%bash
# Create the folder structure
mkdir -p app/templates app/static

# index.html (upload form)
cat > app/templates/index.html << 'EOF'
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>People Counter</title>
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <link rel="stylesheet"
        href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"/>
</head>
<body class="min-h-screen bg-gradient-to-r from-blue-200 via-purple-200 to-pink-200
             flex items-center justify-center p-4">
  <div class="bg-white bg-opacity-80 rounded-2xl p-8 shadow-xl w-full max-w-sm
              animate__animated animate__fadeInUp">
    <h1 class="text-3xl font-bold text-gray-800 text-center mb-6">👥 People Counter</h1>
    {% if error %}
      <p class="text-red-600 text-center mb-4">{{ error }}</p>
    {% endif %}
    <form action="/" method="post" enctype="multipart/form-data" class="space-y-4">
      <input type="file" name="file" accept="image/*" required
             class="block w-full text-gray-700 file:py-2 file:px-4
                    file:rounded-full file:bg-indigo-600 file:text-white
                    hover:file:bg-indigo-700 transition"/>
      <button type="submit"
              class="w-full py-2 bg-indigo-600 hover:bg-indigo-700 text-white font-semibold
                     rounded-full transform hover:scale-105 transition">
        Upload & Detect
      </button>
    </form>
  </div>
</body>
</html>
EOF

# result.html (only annotated image + count)
cat > app/templates/result.html << 'EOF'
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Detection Result</title>
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <link rel="stylesheet"
        href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"/>
</head>
<body class="min-h-screen bg-gradient-to-r from-blue-200 via-purple-200 to-pink-200
             flex items-center justify-center p-4">
  <div class="bg-white bg-opacity-90 rounded-2xl p-8 shadow-xl w-full max-w-md
              animate__animated animate__zoomIn">
    <h2 class="text-2xl font-bold text-gray-800 mb-6">👀 People Detected: {{ count }}</h2>
    <img src="{{ annotated_url }}" alt="Annotated"
         class="mx-auto rounded-lg shadow-md w-full mb-6"/>
    <a href="/" class="inline-block px-6 py-2 bg-indigo-600 hover:bg-indigo-700 text-white
                       font-semibold rounded-full transform hover:rotate-3 transition">
      Try Another
    </a>
  </div>
</body>
</html>
EOF


In [56]:
%%bash
# Write the YOLOv8 detector module
cat > app/people_detector.py << 'EOF'
import cv2, os
from ultralytics import YOLO

# Load YOLOv8-nano model once
model = YOLO('yolov8n.pt')

def detect_people(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return 0, image_path, {}
    results = model(image_path)[0]
    boxes   = results.boxes.xyxy
    classes = results.boxes.cls

    annotated = img.copy()
    count = 0
    for box, cls in zip(boxes, classes):
        if int(cls) == 0:  # person class
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(annotated, (x1, y1), (x2, y2), (0,255,0), 2)
            count += 1

    base = os.path.basename(image_path)
    out_path = os.path.join(os.path.dirname(image_path), 'annotated_' + base)
    cv2.imwrite(out_path, annotated)
    return count, out_path, {}
EOF


In [57]:
%%bash
# Write the Flask app
cat > app/app.py << 'EOF'
from flask import Flask, render_template, request, url_for
from werkzeug.utils import secure_filename
import os
from people_detector import detect_people

app = Flask(__name__, static_folder='static', template_folder='templates')
UPLOAD_FOLDER = os.path.join(app.root_path, 'static')
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/', methods=['GET','POST'])
def index():
    if request.method == 'POST':
        file = request.files.get('file')
        if not file or file.filename == '':
            return render_template('index.html', error='Please choose an image.')
        fname = secure_filename(file.filename)
        in_path = os.path.join(UPLOAD_FOLDER, fname)
        file.save(in_path)

        count, out_path, _ = detect_people(in_path)
        ann_url = url_for('static', filename=os.path.basename(out_path))
        return render_template('result.html', count=count, annotated_url=ann_url)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
EOF


In [58]:
%%bash
# Download & install cloudflared CLI
wget -qO cloudflared.deb \
  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
apt-get update -qq
apt-get install -qq -y ./cloudflared.deb


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [59]:
%%bash
# Start Flask in the background and give it a moment
nohup python app/app.py > flask.log 2>&1 &
sleep 3
echo "🔄 Flask server started; logs → flask.log"


🔄 Flask server started; logs → flask.log


In [60]:
import subprocess, re

# Launch cloudflared tunnel and capture its output
proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:5000"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

public_url = None
for line in proc.stdout:
    if (m := re.search(r"https?://[-\w]+\.trycloudflare\.com", line)):
        public_url = m.group(0)
        break

if public_url:
    print(f"🌐 Public URL → {public_url}")
else:
    print("❌ Failed to retrieve the tunnel URL. Check cloudflared installation.")


🌐 Public URL → https://sewing-counting-southampton-us.trycloudflare.com
